In [115]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('.')
import rr_parser
import pokebase as pb
from rr_parser.enums import GameType

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
with open('C:\\Users\\Jackson Paull\\Dropbox\\Radical Red\\RadRedNUZLOCKE.sav', "rb") as f:
    b = f.read()
    
g = rr_parser.games.RadicalRed(b)
gs = g.game_save
box_data = b''.join([gs.sections[i]._data for i in range(5, 14)])
pc = rr_parser.sections.PC(GameType.RR, gs)

In [122]:
from rr_parser.functions import pkm_set_to_text

In [152]:
print(pkm_set_to_text(pc.boxes[0].pokemon[2], level=15))

Weedle
Level: 15
Relaxed Nature
Ability: Shield Dust
EVs: 0 HP / 0 Atk / 0 Def / 2 Spe / 0 SpA / 0 SpD
IVs: 29 HP / 5 Atk / 3 Def / 8 Spe / 1 SpA / 19 SpD
- String Shot
- Poison Sting



In [132]:
i = 4 # Box slot to look at
pkm_data = pc._data[4+58*i:4+58*(i+1)]
personality_value = pkm_data[:0x4]
ot_id = pkm_data[0x4:0x8]

nickname = pkm_data[0x8:0x12]
nickname = rr_parser.charsets.Gen3Charset.bin2char3(nickname)

lang = pkm_data[0x12]
misc_flags = pkm_data[0x13]
ot_name = pkm_data[0x14:0x1B]
ot_name = rr_parser.charsets.Gen3Charset.bin2char3(ot_name)
markings = pkm_data[0x1B]
substruct_data = pkm_data[0x1C:]

# Growth Substruct
species = int.from_bytes(substruct_data[:2], 'little')
item = int.from_bytes(substruct_data[2:4], 'little')
exp = int.from_bytes(substruct_data[4:8], 'little')
pp_bonus_data = substruct_data[8]
friendship = substruct_data[9]

# Attack Substruct
move_data = int.from_bytes(substruct_data[11:16], 'little')
moves = [
    move_data & 0x3FF,
    move_data >> 10 & 0x3FF,
    move_data >> 20 & 0x3FF,
    move_data >> 30 & 0x3FF,
]

# EVs
EVs = [
    substruct_data[16],
    substruct_data[17],
    substruct_data[18],
    substruct_data[19],
    substruct_data[20],
    substruct_data[21],
]
EVs_dict = dict(zip(
    ['HP', 'Atk', 'Def', 'Spe', 'SpA', 'SpD'], EVs
))

# Misc Substruct


In [135]:
from rr_parser.pkms import EVs

In [134]:
substruct_data

b'\x10\x00\x00\x00\xfb\x07\x00\x00\x00m\x03\x10\x90 FJ\x06\x02\x01\x01\x03\x00\x00e\x03\x02\xf7\xee_9'

In [136]:
EVs(substruct_data[16:22])

// Poison Sting (0x028), String Shot (0x051), None, None
0000 1110  //  0E <-- Must be related to the first 2 bits of all moves, stays constant between all mons with moves with form 0x0??, but changes when pidgey knows feather dance...
0010 1000  //  28 <--- Last 8 bits of first move ID
0100 01/ 00  //  44 <-- xor of 5 and 1, then and of 5 and 1
0000 / 0001  //  01
0000 0000  //  00
0000 0000  //  00


// String Shot, Poison Sting, None, None
0000 1110  // 0E
0101 0001  // 51  <--- Last 8 bits of first move ID
10 1000/ 00  // A0 <-- xor of 2/8, then and of 2/8
0000 / 00 00  // 00
0000 0000  // 00
0000 0000  // 00

// Splash (0x096), None, None, None
0000 1110  // 0E
1001 0110  // 96
0000 0000  // 00
0000 0000  // 00
0000 0000  // 00
0000 0000  // 00

// Growl (0x2D), Tackle (0x21), Ember (0x34), Focus Energy (0x74)
0000 1110 // 0E
0010 1101 // 2D 
10 00 01 /00 // 84 <-- xor then and encryption fails here...
0100 / 00 00 // 40
0000 0011 // 03
0001 1101 // 1D

// Take Down (0x24), Feather Dance (0x129), Gust (0x10), Quick Attack (0x62)
0000 0011 // 03
0010 0100 // 24
1010 01/ 00 // A4
0000/  0100 // 04
10/ 00 0001 // 81
0001 1000 // 18

// Feather Dance, Take Down, Gust, Quick Attack
0000 0011 // 03
0010 1001 // 29
1001 0001 // 91
0000 0000 // 00
1000 0001 // 81
0001 1000 // 18

// Gust, Take Down, Feather Dance, Quick Attack
0000 0011 // 03
0001 0000 // 10
1001 0000 // 90
1001 0000 // 90
1001 0010 // 92
0001 1000 // 18

// Gust, Take Down, Quick Attack, Feather Dance
0000 0011 // 03
1000 0000 // 10
1001 0000 // 90
0010 0000 // 20
0100 0110 // 46
0100 1010 // 4A


In [102]:
moves

[16, 36, 98, 297]